In [42]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt

# optimal parameters

In [43]:
pd.read_json('logs_config', orient = 'index') #Number of parameters = 3283001

,0
depth,5.00
dropout,0.05
ffn_num_layers,2.00
hidden_size,1000.00


In [44]:

train_observed = pd.read_csv('./train_dmpnn.csv').logS
valid_observed = pd.read_csv('./valid_dmpnn.csv').logS
test_observed = pd.read_csv('./test_dmpnn.csv').logS
etc_observed = pd.read_csv('./etc_dmpnn.csv').logS

res = []
for i in [1,2,3]:

    # train
    train_predicted = pd.read_csv('./pred_train_dmpnn_%s.csv' % i).logS
    #valid
    valid_predicted = pd.read_csv('./pred_valid_dmpnn_%s.csv' % i).logS

    #test
    test_predicted = pd.read_csv('./pred_test_dmpnn_%s.csv' % i).logS

    #etc
    etc_predicted = pd.read_csv('./pred_etc_dmpnn_%s.csv' % i).logS

    train_rmse = np.sqrt(mean_squared_error(train_observed, train_predicted))
    valid_rmse = np.sqrt(mean_squared_error(valid_observed, valid_predicted))
    test_rmse = np.sqrt(mean_squared_error(test_observed, test_predicted))
    etc_rmse = np.sqrt(mean_squared_error(etc_observed, etc_predicted))
    
    train_pcc, train_p = pearsonr(train_observed, train_predicted)
    valid_pcc, valid_p = pearsonr(valid_observed, valid_predicted)
    test_pcc, test_p = pearsonr(test_observed, test_predicted)    
    etc_pcc, etc_p = pearsonr(etc_observed, etc_predicted)
    
    res.append([i, train_rmse, valid_rmse, test_rmse, etc_rmse, 
                train_pcc, valid_pcc, test_pcc, etc_pcc,
                train_p, valid_p, test_p, etc_p
               ])

In [46]:
df = pd.DataFrame(res, columns = ['random', 'train_rmse', 'valid_rmse', 'test_rmse', 'etc_rmse',
                            'train_pcc', 'valid_pcc', 'test_pcc', 'etc_pcc',
                             'train_p', 'valid_p', 'test_p', 'etc_p'])

In [48]:
df.mean()

random        2.000000e+00
train_rmse    3.221434e-01
valid_rmse    5.116568e-01
test_rmse     4.054026e-01
etc_rmse      1.008825e+00
train_pcc     9.822910e-01
valid_pcc     9.522916e-01
test_pcc      9.648177e-01
etc_pcc       4.537655e-01
train_p       0.000000e+00
valid_p       1.150249e-58
test_p        5.541903e-66
etc_p         2.335637e-07
dtype: float64

In [59]:
avg = df.mean()
sd = df.std()
res = []
for i in avg.index:
    res.append([i, '%.3f ± %.3f' % (avg.loc[i], sd.loc[i])])

In [68]:
dfr = pd.DataFrame(res).iloc[1:].set_index(0).T

In [70]:
dfr['# trainable params'] = 3283001
dfr['model'] = 'DMPNN'

In [73]:
dfr.to_csv('results_DMPNN.csv')
dfr.to_pickle('results_DMPNN.pkl')